## Ignore Warnings: 
This cell disables warnings to maintain a clean output during execution.

In [85]:
import warnings
warnings.filterwarnings('ignore')

## Import Required Libraries: 
This cell imports the necessary libraries for Grad-CAM

In [86]:
import os
import cv2
import numpy as np
import timm
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

## Set Device: 
This cell checks for GPU availability and sets the device to either cuda or cpu.

In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Define Custom Dataset: 
This cell defines a class for creating a custom dataset for image loading and processing.

In [88]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        for class_dir in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_dir)
            if os.path.isdir(class_path):
                for img_file in os.listdir(class_path):
                    self.image_paths.append((os.path.join(class_path, img_file), class_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, label = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        return image, img_path

## Apply Transformations: 
This cell sets up transformations for test data preprocessing.

In [89]:
test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Load The Dataset: 
Loading the images from the test dataset 

In [90]:
test_dir = r"C:\Users\jamee\Favorites\Downloads\GradCam"
test_dataset = CustomImageDataset(test_dir, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(f"Loaded {len(test_dataset)} test images.")

Loaded 512 test images.


## Load Swin Transformer model:


In [91]:
num_classes = len(os.listdir(test_dir))
model = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load("swin_skin_disease_model_with3.pth", map_location=device))
model = model.to(device)
model.eval()
print("Model loaded successfully!")

Model loaded successfully!


# Define the reshape transform for Swin Transformer compatibility

In [92]:
def reshape_transform(tensor):
    if len(tensor.shape) == 3:  # Handle Swin Transformer activations
        batch_size, num_patches, num_channels = tensor.shape
        height = width = int(num_patches ** 0.5)  # Assuming square patches
        tensor = tensor.permute(0, 2, 1).contiguous()  # (B, C, Patches)
        tensor = tensor.view(batch_size, num_channels, height, width)  # Reshape to (B, C, H, W)
    return tensor

## Initialize Model: 
This cell loads a pre-trained model and prepares it for Grad-CAM visualization.

In [93]:
target_layers = [model.layers[-1].blocks[-1].attn]  # Experiment with `attn` layer for better results
cam = GradCAM(model=model, target_layers=target_layers, reshape_transform=reshape_transform)

# Output directory for visualizations
output_dir = os.path.join(test_dir, "grad_cam_visualizations")
os.makedirs(output_dir, exist_ok=True)
print("Grad-CAM initialized successfully!")

Grad-CAM initialized successfully!


## Save Results: 
This cell saves the Grad-CAM results as output images in a specified directory.
## Evaluate Model Performance: 
This cell evaluates the model’s performance using the provided dataset and metrics.

In [94]:
for i, (image_tensor, img_path) in enumerate(test_loader):
    image_tensor = image_tensor.to(device)
    img_path = img_path[0]  # Extract single image path

    # Generate Grad-CAM heatmap
    targets = [ClassifierOutputTarget(0)]  # Replace 0 with your desired class index if needed
    grayscale_cam = cam(input_tensor=image_tensor, targets=targets)[0]  # [0] for single image

    # Normalize the grayscale CAM to remove noise
    grayscale_cam = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min())

    # Load the original image for visualization
    original_image = cv2.imread(img_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image = cv2.resize(original_image, (224, 224))
    original_image = np.float32(original_image) / 255.0

    # Overlay Grad-CAM heatmap on the original image
    heatmap = show_cam_on_image(original_image, grayscale_cam, use_rgb=True)

    # Save the visualization
    output_path = os.path.join(output_dir, f"grad_cam_{i + 1}.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(heatmap, cv2.COLOR_RGB2BGR))
    print(f"Saved Grad-CAM visualization for image {i + 1} at {output_path}.")

Saved Grad-CAM visualization for image 1 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_1.jpg.
Saved Grad-CAM visualization for image 2 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_2.jpg.
Saved Grad-CAM visualization for image 3 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_3.jpg.
Saved Grad-CAM visualization for image 4 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_4.jpg.
Saved Grad-CAM visualization for image 5 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_5.jpg.
Saved Grad-CAM visualization for image 6 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_6.jpg.
Saved Grad-CAM visualization for image 7 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\grad_cam_7.jpg.
Saved Grad-CAM visualization for image 8 at C:\Users\jamee\Favorites\Downloads\GradCam\grad_cam_visualizations\

In [95]:
# for class_index in range(24):  # Loop through all 24 classes (0 to 23)
#     for i, (image_tensor, img_path) in enumerate(test_loader):
#         image_tensor = image_tensor.to(device)
#         img_path = img_path[0]  # Extract single image path

#         # Generate Grad-CAM heatmap for the current class
#         targets = [ClassifierOutputTarget(class_index)]  
#         grayscale_cam = cam(input_tensor=image_tensor, targets=targets)[0]  # [0] for single image

#         # Normalize the grayscale CAM to remove noise
#         grayscale_cam = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min())

#         # Load the original image for visualization
#         original_image = cv2.imread(img_path)
#         original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
#         original_image = cv2.resize(original_image, (224, 224))
#         original_image = np.float32(original_image) / 255.0

#         # Overlay Grad-CAM heatmap on the original image
#         heatmap = show_cam_on_image(original_image, grayscale_cam, use_rgb=True)

#         # Convert both images to BGR format for OpenCV display
#         original_image_bgr = (original_image * 255).astype(np.uint8)
#         heatmap_bgr = (heatmap * 255).astype(np.uint8)

#         # Combine the original image and Grad-CAM heatmap side by side
#         combined_image = cv2.hconcat([original_image_bgr, heatmap_bgr])

#         # Save the combined image
#         output_path = os.path.join(output_dir, f"class_{class_index}_grad_cam_{i + 1}.jpg")
#         cv2.imwrite(output_path, combined_image)
#         print(f"Saved combined Grad-CAM visualization for image {i + 1} with class index {class_index} at {output_path}.")